In [2]:
import requests

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# Metro Manila (National Capital Region) on OSM has Wikidata id Q13580
# Using the wikidata tag is robust against name variants like "NCR" / "National Capital Region"
query = """
[out:json][timeout:180];

// Find the Metro Manila administrative area
area[wikidata="Q13580"]->.mm;

// Count restaurants mapped as nodes
node["amenity"="restaurant"](area.mm);
out count;

// Count restaurants mapped as ways (building footprints/areas)
way["amenity"="restaurant"](area.mm);
out count;

// Count restaurants mapped as relations (rare, but include for completeness)
relation["amenity"="restaurant"](area.mm);
out count;
"""

r = requests.get(OVERPASS_URL, params={"data": query})
r.raise_for_status()
data = r.json()

def extract_count(count_elem, prefer=None):
    """
    Overpass 'out count' returns a 'count' element.
    Depending on the selector, the number might be in tags['count']
    or in tags like 'nodes'/'ways'/'relations'. Handle both.
    """
    tags = count_elem.get("tags", {})
    if "count" in tags:
        return int(tags["count"])
    # fallback patterns seen in some responses
    for key in (prefer or []):
        if key in tags:
            return int(tags[key])
    # generic fallback if the server returned a breakdown
    for k in ("nodes", "ways", "relations", "total"):
        if k in tags:
            return int(tags[k])
    return 0

# Expect three count elements in the same order as the three blocks above
node_count   = extract_count(data["elements"][0], prefer=["nodes"])
way_count    = extract_count(data["elements"][1], prefer=["ways"])
rel_count    = extract_count(data["elements"][2], prefer=["relations"])
total = node_count + way_count + rel_count

print({
    "metro_manila_restaurants": {
        "nodes": node_count,
        "ways": way_count,
        "relations": rel_count,
        "total": total
    }
})


{'metro_manila_restaurants': {'nodes': 4599, 'ways': 396, 'relations': 4, 'total': 4999}}


In [ ]:
import requests
import time

OVERPASS_URL = "https://overpass-api.de/api/interpreter"
UA = "osm-metro-manila-5yr/1.0 (+https://example.com; you@example.com)"  # <- add your contact

def extract_count(count_elem, prefer=None):
    tags = count_elem.get("tags", {})
    if "count" in tags:
        return int(tags["count"])
    for key in (prefer or []):
        if key in tags:
            return int(tags[key])
    for k in ("nodes", "ways", "relations", "total"):
        if k in tags:
            return int(tags[k])
    return 0

def run_count_at_date(date_iso: str):
    """
    Query Metro Manila (wikidata=Q13580) at a specific snapshot date.
    Returns dict with nodes/ways/relations/total.
    """
    date_clause = f'[date:"{date_iso}"]' if date_iso else ''
    query = f"""
    [out:json][timeout:180]{date_clause};

    // Find the Metro Manila administrative area
    area[wikidata="Q13580"]->.mm;

    // Count restaurants mapped as nodes
    node["amenity"="restaurant"](area.mm);
    out count;

    // Count restaurants mapped as ways (building footprints/areas)
    way["amenity"="restaurant"](area.mm);
    out count;

    // Count restaurants mapped as relations (rare, but include for completeness)
    relation["amenity"="restaurant"](area.mm);
    out count;
    """
    r = requests.get(OVERPASS_URL, params={"data": query}, headers={"User-Agent": UA}, timeout=300)
    r.raise_for_status()
    data = r.json()

    node_count = extract_count(data["elements"][0], prefer=["nodes"])
    way_count  = extract_count(data["elements"][1], prefer=["ways"])
    rel_count  = extract_count(data["elements"][2], prefer=["relations"])
    return {
        "nodes": node_count,
        "ways": way_count,
        "relations": rel_count,
        "total": node_count + way_count + rel_count
    }

# ---- Collect current + past 5 complete years (2020–2024) -------------------
# Change these if you prefer different cutoffs (e.g., mid-year snapshots).
snapshot_dates = [
    "2024-12-31T23:59:59Z",
    "2023-12-31T23:59:59Z",
    "2022-12-31T23:59:59Z",
    "2021-12-31T23:59:59Z",
    "2020-12-31T23:59:59Z",
]
# Optional: also include "current" live data (comment out if you don’t want it)
include_current = True

results_by_date = {}

# Current (no [date:] clause)
if include_current:
    counts = run_count_at_date(None)
    results_by_date["current"] = counts
    time.sleep(0.7)  # be polite to the API

# Historical snapshots
for d in snapshot_dates:
    try:
        counts = run_count_at_date(d)
    except Exception as e:
        counts = {"nodes": None, "ways": None, "relations": None, "total": None, "error": str(e)}
    results_by_date[d[:10]] = counts  # use YYYY-MM-DD as the key
    time.sleep(0.7)

# Pretty print JSON-ish summary
print({"metro_manila_restaurants": results_by_date})

{'metro_manila_restaurants': {'current': {'nodes': 4599, 'ways': 396, 'relations': 4, 'total': 4999}, '2024-12-31': {'nodes': 4398, 'ways': 391, 'relations': 4, 'total': 4793}, '2023-12-31': {'nodes': 3859, 'ways': 373, 'relations': 2, 'total': 4234}, '2022-12-31': {'nodes': 3565, 'ways': 351, 'relations': 0, 'total': 3916}, '2021-12-31': {'nodes': 3439, 'ways': 342, 'relations': 0, 'total': 3781}, '2020-12-31': {'nodes': 3311, 'ways': 314, 'relations': 0, 'total': 3625}}}


In [8]:
# ---- Optional: tabular output with pandas -----------------------------------
# Uncomment this block if you want a DataFrame
import pandas as pd
rows = []
for snapshot, vals in results_by_date.items():
    row = {"snapshot": snapshot}
    row.update(vals)
    rows.append(row)
df = pd.DataFrame(rows).sort_values("snapshot")
print(df)
df.to_csv("metro_manila_restaurants_5yr.csv", index=False)

     snapshot  nodes  ways  relations  total
5  2020-12-31   3311   314          0   3625
4  2021-12-31   3439   342          0   3781
3  2022-12-31   3565   351          0   3916
2  2023-12-31   3859   373          2   4234
1  2024-12-31   4398   391          4   4793
0     current   4599   396          4   4999


In [9]:
import requests
import time
from datetime import datetime

OVERPASS_URL = "https://overpass-api.de/api/interpreter"
UA = "osm-metro-manila-5yr/1.0 (+https://example.com; you@example.com)"  # <- add your contact

# ---------- existing helpers (unchanged) ----------
def extract_count(count_elem, prefer=None):
    tags = count_elem.get("tags", {})
    if "count" in tags:
        return int(tags["count"])
    for key in (prefer or []):
        if key in tags:
            return int(tags[key])
    for k in ("nodes", "ways", "relations", "total"):
        if k in tags:
            return int(tags[k])
    return 0

def run_count_at_date(date_iso: str):
    """
    Query Metro Manila (wikidata=Q13580) at a specific snapshot date.
    Returns dict with nodes/ways/relations/total.
    """
    date_clause = f'[date:"{date_iso}"]' if date_iso else ''
    query = f"""
    [out:json][timeout:180]{date_clause};

    // Find the Metro Manila administrative area
    area[wikidata="Q13580"]->.mm;

    // Count restaurants mapped as nodes
    node["amenity"="restaurant"](area.mm);
    out count;

    // Count restaurants mapped as ways (building footprints/areas)
    way["amenity"="restaurant"](area.mm);
    out count;

    // Count restaurants mapped as relations (rare, but include for completeness)
    relation["amenity"="restaurant"](area.mm);
    out count;
    """
    r = requests.get(OVERPASS_URL, params={"data": query}, headers={"User-Agent": UA}, timeout=300)
    r.raise_for_status()
    data = r.json()

    node_count = extract_count(data["elements"][0], prefer=["nodes"])
    way_count  = extract_count(data["elements"][1], prefer=["ways"])
    rel_count  = extract_count(data["elements"][2], prefer=["relations"])
    return {
        "nodes": node_count,
        "ways": way_count,
        "relations": rel_count,
        "total": node_count + way_count + rel_count
    }

# ===================== EDIT HERE (new): Wikidata helpers =====================
WD_SPARQL = "https://query.wikidata.org/sparql"
WD_HEADERS = {
    "User-Agent": UA,
    "Accept": "application/sparql-results+json"
}

def get_area_km2_q13580():
    """
    Returns area in km^2 from Wikidata (P2046) for Q13580 (Metro Manila).
    If multiple area values exist, picks the largest numeric (common on WD).
    """
    q = """
    SELECT ?area WHERE {
      wd:Q13580 wdt:P2046 ?area.
    }
    """
    r = requests.get(WD_SPARQL, params={"query": q}, headers=WD_HEADERS, timeout=60)
    r.raise_for_status()
    res = r.json()["results"]["bindings"]
    if not res:
        return None
    vals = [float(x["area"]["value"]) for x in res]
    # P2046 is in square metres — convert to km^2 if needed
    # On WD, unit may be encoded; safest: attempt to detect unit via explicit query.
    # Simpler approach: fetch with unit. Let's do a unit-aware query instead:

    # (We’ll re-query with unit to be accurate)
    q2 = """
    SELECT ?area ?unitLabel WHERE {
      wd:Q13580 p:P2046 ?stmt.
      ?stmt ps:P2046 ?area.
      OPTIONAL { ?stmt psn:P2046 ?areaNode. ?areaNode wikibase:quantityUnit ?unit. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """
    r2 = requests.get(WD_SPARQL, params={"query": q2}, headers=WD_HEADERS, timeout=60)
    r2.raise_for_status()
    res2 = r2.json()["results"]["bindings"]
    best_km2 = None
    for row in res2:
        val = float(row["area"]["value"])
        unit = row.get("unitLabel", {}).get("value", "")
        # Common units: square kilometre, square metre
        if "square kilometre" in unit or "square kilometer" in unit:
            km2 = val
        elif "square metre" in unit or "square meter" in unit:
            km2 = val / 1_000_000.0
        else:
            # fallback assume m^2
            km2 = val / 1_000_000.0
        if best_km2 is None or km2 > best_km2:
            best_km2 = km2
    return best_km2

def get_population_q13580_for_year(year: int):
    """
    Returns the best population value for Q13580 (Metro Manila) whose
    point-in-time (P585) is <= Dec 31 of the requested year.
    Falls back to the latest available if none found before that year.
    """
    # Find the latest population with P585 <= target date
    target = f"{year}-12-31T23:59:59Z"
    q = f"""
    SELECT ?pop ?when WHERE {{
      wd:Q13580 p:P1082 ?stmt.
      ?stmt ps:P1082 ?pop.
      OPTIONAL {{ ?stmt pq:P585 ?when. }}
    }}
    """
    r = requests.get(WD_SPARQL, params={"query": q}, headers=WD_HEADERS, timeout=60)
    r.raise_for_status()
    rows = r.json()["results"]["bindings"]
    if not rows:
        return None
    # pick the row with the latest date <= target; otherwise the overall latest
    def row_key(row):
        when = row.get("when", {}).get("value")
        if when:
            try:
                return datetime.fromisoformat(when.replace("Z", "+00:00"))
            except Exception:
                return None
        return None

    # Separate those with dates from those without
    with_dates = [row for row in rows if row.get("when")]
    without_dates = [row for row in rows if not row.get("when")]

    target_dt = datetime.fromisoformat(target.replace("Z", "+00:00"))
    candidates = []
    for row in with_dates:
        dt = row_key(row)
        if dt and dt <= target_dt:
            candidates.append((dt, float(row["pop"]["value"])))

    if candidates:
        # Latest at-or-before target
        candidates.sort(key=lambda x: x[0], reverse=True)
        return candidates[0][1]

    # Fallback: take the overall latest dated value, else any undated value
    if with_dates:
        with_dates.sort(key=lambda r: row_key(r) or datetime.min, reverse=True)
        return float(with_dates[0]["pop"]["value"])
    # Last resort: any undated
    return float(without_dates[0]["pop"]["value"]) if without_dates else None
# ===================== END EDITS (new): Wikidata helpers =====================

# ---- Collect current + past 5 complete years (2020–2024) -------------------
snapshot_dates = [
    "2024-12-31T23:59:59Z",
    "2023-12-31T23:59:59Z",
    "2022-12-31T23:59:59Z",
    "2021-12-31T23:59:59Z",
    "2020-12-31T23:59:59Z",
]
include_current = True

results_by_date = {}

# ===================== EDIT HERE: fetch area once and reuse ==================
area_km2 = get_area_km2_q13580()  # may be None if WD lacks it
# ============================================================================

# Current (no [date:] clause)
if include_current:
    counts = run_count_at_date(None)
    # ===================== EDIT HERE: add population & density ==============
    pop = get_population_q13580_for_year(datetime.utcnow().year)
    density = (pop / area_km2) if (pop is not None and area_km2) else None
    counts.update({"population": pop, "area_km2": area_km2, "population_density_per_km2": density})
    # =========================================================================
    results_by_date["current"] = counts
    time.sleep(0.7)

# Historical snapshots
for d in snapshot_dates:
    try:
        counts = run_count_at_date(d)
        # ===================== EDIT HERE: add population & density ==========
        yr = int(d[:4])
        pop = get_population_q13580_for_year(yr)
        density = (pop / area_km2) if (pop is not None and area_km2) else None
        counts.update({"population": pop, "area_km2": area_km2, "population_density_per_km2": density})
        # ====================================================================
    except Exception as e:
        counts = {"nodes": None, "ways": None, "relations": None, "total": None, "error": str(e),
                  "population": None, "area_km2": area_km2, "population_density_per_km2": None}
    results_by_date[d[:10]] = counts
    time.sleep(0.7)

print({"metro_manila_restaurants": results_by_date})


C:\Users\Enzo\AppData\Local\Temp\ipykernel_97092\3169810055.py:188: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  pop = get_population_q13580_for_year(datetime.utcnow().year)


{'metro_manila_restaurants': {'current': {'nodes': 4599, 'ways': 396, 'relations': 4, 'total': 4999, 'population': 14001751.0, 'area_km2': 0.00061139, 'population_density_per_km2': 22901504767.82414}, '2024-12-31': {'nodes': 4398, 'ways': 391, 'relations': 4, 'total': 4793, 'population': 14001751.0, 'area_km2': 0.00061139, 'population_density_per_km2': 22901504767.82414}, '2023-12-31': {'nodes': 3859, 'ways': 373, 'relations': 2, 'total': 4234, 'population': 13484462.0, 'area_km2': 0.00061139, 'population_density_per_km2': 22055417981.9755}, '2022-12-31': {'nodes': 3565, 'ways': 351, 'relations': 0, 'total': 3916, 'population': 13484462.0, 'area_km2': 0.00061139, 'population_density_per_km2': 22055417981.9755}, '2021-12-31': {'nodes': 3439, 'ways': 342, 'relations': 0, 'total': 3781, 'population': 13484462.0, 'area_km2': 0.00061139, 'population_density_per_km2': 22055417981.9755}, '2020-12-31': {'nodes': 3311, 'ways': 314, 'relations': 0, 'total': 3625, 'population': 13484462.0, 'area_

In [11]:
import pandas as pd
rows = []
for snap, vals in results_by_date.items():
    row = {"snapshot": snap}
    row.update(vals)
    rows.append(row)
df = pd.DataFrame(rows).sort_values("snapshot")
df

,snapshot,nodes,ways,relations,total,population,area_km2,population_density_per_km2
5,2020-12-31,3311,314,0,3625,13484462.0,0.000611,2.205542e+10
4,2021-12-31,3439,342,0,3781,13484462.0,0.000611,2.205542e+10
3,2022-12-31,3565,351,0,3916,13484462.0,0.000611,2.205542e+10
2,2023-12-31,3859,373,2,4234,13484462.0,0.000611,2.205542e+10
1,2024-12-31,4398,391,4,4793,14001751.0,0.000611,2.290150e+10
0,current,4599,396,4,4999,14001751.0,0.000611,2.290150e+10


In [15]:
import requests, json
from datetime import datetime

UA = "manila-pop-sample/0.1 laltonaga.MSDS2026@aim.edu"

# 👉 EDIT this to the EXACT PxWeb table that has annual projections by Province/HUC.
# (Example path under Population & Housing; use the table page then switch /pxweb/... to /api/...)
BASE = "https://openstat.psa.gov.ph/PXWeb/api/en/DB/DB__1A__PO"
TABLE = "0021A3BPOP1.px"  # e.g. “Projected Population Based on 2020 CPH … Single-Year Interval”
API = f"{BASE}/{TABLE}"

# 1) Discover variables/codes in the table
meta = requests.get(API, headers={"User-Agent": UA, "Accept":"application/json"}).json()
# Find the right variable codes from meta (print(meta) once to see exact names in your table)
# Common patterns in PxWeb:
#  - a geography code (e.g., "Region/Province/Highly Urbanized City" or "PSGC")
#  - a time/year code (e.g., "Year")
#  - a sex/age code (set to “Both Sexes”, “All Ages” when you want totals)

# 2) Build a POST to pull City of Manila projections for the last 5 calendar years
years = [str(y) for y in range(datetime.utcnow().year - 5 + 1, datetime.utcnow().year + 1)]
body = {
  "query": [
    # ⚠️ Replace "Geo" and value below using the actual code/value from meta for City of Manila.
    {"code": "Geo", "selection": {"filter": "item", "values": ["City of Manila"]}},
    # If the table has Sex/Age variables, set them to totals:
    # {"code": "Sex", "selection": {"filter": "item", "values": ["Both Sexes"]}},
    # {"code": "Age", "selection": {"filter": "item", "values": ["All Ages"]}},
    {"code": "Year", "selection": {"filter": "item", "values": years}}
  ],
  "response": {"format": "JSON"}
}

res = requests.post(API, json=body, headers={"User-Agent": UA, "Accept":"application/json"})
res.raise_for_status()
data = res.json()

# 3) Tidy to rows (year, population)
rows = []
for cell, stub in zip(data["data"], data["dimension"]["Year"]["category"]["index"].items()):
    # map index back to year string
    year_code = next(k for k,v in data["dimension"]["Year"]["category"]["index"].items() if v == len(rows))
    year_label = data["dimension"]["Year"]["category"]["label"][year_code]
    rows.append({"year": year_label, "population": cell["value"]})

print(rows)

# Optional: pandas DataFrame
# import pandas as pd
# df = pd.DataFrame(rows).sort_values("year")
# print(df)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests
import time
from datetime import datetime

OVERPASS_URL = "https://overpass-api.de/api/interpreter"
UA = "osm-metro-manila-5yr/1.0 (+https://example.com; you@example.com)"  # <- add your contact

# ---------- existing helpers (unchanged) ----------
def extract_count(count_elem, prefer=None):
    tags = count_elem.get("tags", {})
    if "count" in tags:
        return int(tags["count"])
    for key in (prefer or []):
        if key in tags:
            return int(tags[key])
    for k in ("nodes", "ways", "relations", "total"):
        if k in tags:
            return int(tags[k])
    return 0

def run_count_at_date(date_iso: str):
    """
    Query Metro Manila (wikidata=Q13580) at a specific snapshot date.
    Returns dict with nodes/ways/relations/total.
    """
    date_clause = f'[date:"{date_iso}"]' if date_iso else ''
    query = f"""
    [out:json][timeout:180]{date_clause};

    // Find the Metro Manila administrative area
    area[wikidata="Q13580"]->.mm;

    // Count restaurants mapped as nodes
    node["amenity"="restaurant"](area.mm);
    out count;

    // Count restaurants mapped as ways (building footprints/areas)
    way["amenity"="restaurant"](area.mm);
    out count;

    // Count restaurants mapped as relations (rare, but include for completeness)
    relation["amenity"="restaurant"](area.mm);
    out count;
    """
    r = requests.get(OVERPASS_URL, params={"data": query}, headers={"User-Agent": UA}, timeout=300)
    r.raise_for_status()
    data = r.json()

    node_count = extract_count(data["elements"][0], prefer=["nodes"])
    way_count  = extract_count(data["elements"][1], prefer=["ways"])
    rel_count  = extract_count(data["elements"][2], prefer=["relations"])
    return {
        "nodes": node_count,
        "ways": way_count,
        "relations": rel_count,
        "total": node_count + way_count + rel_count
    }

# ===================== NEW: population from OSM via Overpass =================
def _parse_int_or_none(s):
    # Accept "13,484,462", "13484462", "13.5 million" (very rare), return int or None
    if s is None:
        return None
    s = str(s).strip().lower()
    # common case: digits + commas
    digits = ''.join(ch for ch in s if ch.isdigit())
    if digits:
        try:
            return int(digits)
        except Exception:
            pass
    # crude support for "x million"
    if "million" in s:
        try:
            num = float(s.split("million")[0].strip())
            return int(num * 1_000_000)
        except Exception:
            return None
    return None

def get_osm_population_for_snapshot(date_iso: str | None):
    """
    Reads the 'population' tag directly from the Metro Manila boundary relation in OSM.
    If 'date_iso' is provided, uses Overpass attic data at that snapshot.
    Returns an integer or None if missing/unparseable.
    """
    date_clause = f'[date:"{date_iso}"]' if date_iso else ''
    q = f"""
    [out:json][timeout:120]{date_clause};
    relation[wikidata="Q13580"];
    out tags;
    """
    r = requests.get(OVERPASS_URL, params={"data": q}, headers={"User-Agent": UA}, timeout=180)
    r.raise_for_status()
    j = r.json()
    if not j.get("elements"):
        return None
    tags = j["elements"][0].get("tags", {})
    # Preferred: 'population'
    pop = tags.get("population")
    # Some mappers may put country-specific keys e.g. 'population:date', 'census:date' — not counts.
    # If 'population' missing, there isn't a reliable alternative key with the actual count.
    return _parse_int_or_none(pop)
# ===================== END NEW: population from OSM ==========================

# ===================== (kept) Wikidata area helper ===========================
# If you want area from OSM geometry instead, I can swap this to a Shapely-based computation.
WD_SPARQL = "https://query.wikidata.org/sparql"
WD_HEADERS = {"User-Agent": UA, "Accept": "application/sparql-results+json"}

def get_area_km2_q13580():
    q2 = """
    SELECT ?area ?unitLabel WHERE {
      wd:Q13580 p:P2046 ?stmt.
      ?stmt ps:P2046 ?area.
      OPTIONAL { ?stmt psn:P2046 ?areaNode. ?areaNode wikibase:quantityUnit ?unit. }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """
    r2 = requests.get(WD_SPARQL, params={"query": q2}, headers=WD_HEADERS, timeout=60)
    r2.raise_for_status()
    res2 = r2.json()["results"]["bindings"]
    best_km2 = None
    for row in res2:
        val = float(row["area"]["value"])
        unit = row.get("unitLabel", {}).get("value", "")
        if "square kilometre" in unit or "square kilometer" in unit:
            km2 = val
        elif "square metre" in unit or "square meter" in unit:
            km2 = val / 1_000_000.0
        else:
            km2 = val / 1_000_000.0
        if best_km2 is None or km2 > best_km2:
            best_km2 = km2
    return best_km2
# ============================================================================

# ---- Collect current + past 5 complete years (2020–2024) -------------------
snapshot_dates = [
    "2024-12-31T23:59:59Z",
    "2023-12-31T23:59:59Z",
    "2022-12-31T23:59:59Z",
    "2021-12-31T23:59:59Z",
    "2020-12-31T23:59:59Z",
]
include_current = True

results_by_date = {}

# Fetch area once (still from WD here; only population is from OSM)
area_km2 = get_area_km2_q13580()

# Current (no [date:] clause)
if include_current:
    counts = run_count_at_date(None)
    pop = get_osm_population_for_snapshot(None)  # ← population from OSM tag
    density = (pop / area_km2) if (pop is not None and area_km2) else None
    counts.update({"population": pop, "area_km2": area_km2, "population_density_per_km2": density})
    results_by_date["current"] = counts
    time.sleep(0.7)

# Historical snapshots
for d in snapshot_dates:
    try:
        counts = run_count_at_date(d)
        yr = int(d[:4])
        pop = get_osm_population_for_snapshot(d)  # ← OSM snapshot value at that date
        density = (pop / area_km2) if (pop is not None and area_km2) else None
        counts.update({"population": pop, "area_km2": area_km2, "population_density_per_km2": density})
    except Exception as e:
        counts = {"nodes": None, "ways": None, "relations": None, "total": None, "error": str(e),
                  "population": None, "area_km2": area_km2, "population_density_per_km2": None}
    results_by_date[d[:10]] = counts
    time.sleep(0.7)

print({"metro_manila_restaurants": results_by_date})


{'metro_manila_restaurants': {'current': {'nodes': 4599, 'ways': 396, 'relations': 4, 'total': 4999, 'population': 13484462, 'area_km2': 0.00061139, 'population_density_per_km2': 22055417981.9755}, '2024-12-31': {'nodes': 4398, 'ways': 391, 'relations': 4, 'total': 4793, 'population': 13484462, 'area_km2': 0.00061139, 'population_density_per_km2': 22055417981.9755}, '2023-12-31': {'nodes': 3859, 'ways': 373, 'relations': 2, 'total': 4234, 'population': 13484462, 'area_km2': 0.00061139, 'population_density_per_km2': 22055417981.9755}, '2022-12-31': {'nodes': 3565, 'ways': 351, 'relations': 0, 'total': 3916, 'population': None, 'area_km2': 0.00061139, 'population_density_per_km2': None}, '2021-12-31': {'nodes': None, 'ways': None, 'relations': None, 'total': None, 'error': '504 Server Error: Gateway Timeout for url: https://overpass-api.de/api/interpreter?data=%0A++++%5Bout%3Ajson%5D%5Btimeout%3A120%5D%5Bdate%3A%222021-12-31T23%3A59%3A59Z%22%5D%3B%0A++++relation%5Bwikidata%3D%22Q13580%22

In [18]:
pd.DataFrame(results_by_date)

,current,2024-12-31,2023-12-31,2022-12-31,2021-12-31,2020-12-31
nodes,4.599000e+03,4.398000e+03,3.859000e+03,3565.000000,None,3311.000000
ways,3.960000e+02,3.910000e+02,3.730000e+02,351.000000,None,314.000000
relations,4.000000e+00,4.000000e+00,2.000000e+00,0.000000,None,0.000000
total,4.999000e+03,4.793000e+03,4.234000e+03,3916.000000,None,3625.000000
population,1.348446e+07,1.348446e+07,1.348446e+07,NaN,None,NaN
area_km2,6.113900e-04,6.113900e-04,6.113900e-04,0.000611,0.000611,0.000611
population_density_per_km2,2.205542e+10,2.205542e+10,2.205542e+10,NaN,None,NaN
error,NaN,NaN,NaN,NaN,504 Server Error: Gateway Timeout for url: htt...,NaN


In [20]:
data=pd.read_csv("backcasted-values-2010.csv")
data

,Locality,Population,Population density,Accommodation,Agricultural/Plant production,"Art, music, hobbies",Cars,Civic/Amenity,"Clothing, shoes, accessories",Commercial,...,Other Buildings,Other Landuse Key Values,"Outdoors and sport, vehicles",Public transport,Religious,Sport,"Stationery, gifts, books, newspapers",Sustenance,Tourism,Transportation
0,"ABRA, BANGUED",43936.0,355.346234,80.0,0.0,0.0,0.0,9.0,0.0,0.0,...,20.0,6.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0
1,"ABRA, BOLINEY",4063.0,22.247337,14.0,0.0,0.0,0.0,1.0,0.0,0.0,...,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"ABRA, BUCAY",17126.0,164.074533,41.0,0.0,0.0,0.0,3.0,0.0,0.0,...,3.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,"ABRA, BUCLOC",2176.0,42.444203,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"ABRA, DAGUIOMAN",1715.0,17.631504,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,"ZAMBOANGA SIBUGAY, ROSELLER LIM",37216.0,136.460107,71.0,0.0,0.0,0.0,7.0,0.0,0.0,...,7.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
1623,"ZAMBOANGA SIBUGAY, SIAY",38229.0,204.653434,72.0,0.0,0.0,0.0,8.0,0.0,1.0,...,7.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
1624,"ZAMBOANGA SIBUGAY, TALUSAN",25268.0,498.354686,54.0,0.0,0.0,0.0,5.0,0.0,0.0,...,13.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0
1625,"ZAMBOANGA SIBUGAY, TITAY",46456.0,262.833923,83.0,0.0,0.0,0.0,9.0,0.0,0.0,...,15.0,6.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,4.0
